In [1]:
from datetime import datetime, timedelta
from tkinter import messagebox
import tkinter as tk
import pandas as pd
import numpy as np
import locale
import warnings
import os

locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
warnings.filterwarnings('ignore')

hoy = datetime.now()
año = hoy.year
mes = hoy.month

mes_nombre = hoy.strftime('%b').upper()[:3] # ENE
mes_año = f'{mes_nombre}{str(año)[2:]}' # ENE24
año_mes = f'{año}{str(mes).zfill(2)}' # 202401

In [ ]:
def get_file_name() -> list[str]:
    file_list = []
    for file in os.listdir(f'bases/pagos/{año_mes}'):
        if file.endswith('.xlsx') and 'Base Pagos' in file:
            file_list.append(file)
    return file_list

def get_last_date(dates_list: list[datetime]) -> datetime:
    last_date = dates_list[0]
    for date in dates_list:
        if date > last_date:
            last_date = date    
    return last_date

def get_date(file_list: list[str]) -> str:
    dates_list = []
    for file in file_list:
        fecha_list =  file.split(' ')[-1].split('.')[:3]
        dates_list.append(datetime.strptime(f'{fecha_list[0]}.{fecha_list[1]}.{fecha_list[2]}', '%d.%m.%Y'))
    return get_last_date(dates_list).strftime('%d.%m.%Y')

fecha = get_date(get_file_name())
fecha

In [ ]:
def validar_o_crear_carpeta() -> str:
    fecha_hoy = datetime.today().strftime('%Y.%m.%d')
    
    carpeta_reporte = 'reporte'
    
    carpeta_hoy = os.path.join(carpeta_reporte, fecha_hoy)
    
    if not os.path.exists(carpeta_hoy):
        os.makedirs(carpeta_hoy)
        print(f'Carpeta creada: {carpeta_hoy}')
    else:
        print(f'La carpeta ya existe: {carpeta_hoy}')
    
    if not os.path.exists(f'{carpeta_hoy}/multi_agencias'):
        os.makedirs(f'{carpeta_hoy}/multi_agencias')
    else:
        print(f'La carpeta ya existe: {carpeta_hoy}\multi_agencias')
    
    return carpeta_hoy

file_path = validar_o_crear_carpeta()
file_path

In [ ]:
base_pagos_path = f'bases/pagos/{año_mes}/Base Pagos {fecha}.xlsx'
asignacion_path = f'bases/asignacion/{año_mes}/base_asignacion_{mes_año}.xlsx'

print(base_pagos_path)
print(asignacion_path)

monoproducto = f'{file_path}/MONOPRODUCTO {fecha}.xlsx'
multiproducto = f'{file_path}/MULTIPRODUCTO {fecha}.xlsx'
reactiva = f'{file_path}/REACTIVA {fecha}.xlsx'
no_enviados = f'{file_path}/NO ENVIADOS {fecha}.xlsx'
si_enviados = f'{file_path}/ENVIADOS {fecha}.xlsx'

mono_path = os.path.abspath(monoproducto)
multi_path = os.path.abspath(multiproducto)
react_path = os.path.abspath(reactiva)
no_enviados_path = os.path.abspath(no_enviados)
enviados_path = os.path.abspath(si_enviados)

print(mono_path)
print(multi_path)
print(react_path)
print(no_enviados_path)
print(enviados_path)

In [11]:
#################### BASE DE PAGOS ####################

In [ ]:
df_base = pd.read_excel(base_pagos_path)
print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

In [ ]:
fecha_formateada = pd.to_datetime('today').strftime('%d-%b')
fecha_formateada = fecha_formateada[:3] + fecha_formateada[3:].capitalize()

df_base['FECHA_ENVIO'] = fecha_formateada.replace('.', '')
df_base['ID_RESPONSABLE'] = 'MIV'

cols_base = ['FECHA', 'CODCEN', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_base = df_base[cols_base]
df_base.rename(columns={'CODCEN': 'CC'}, inplace=True)

df_base['CC'] = df_base['CC'].astype('Int64').astype(str).str.zfill(8)
df_base['CC'] = df_base['CC'].str.replace(' ', '').str.replace(r'\D', '', regex=True).str[-8:]
df_base['CC'] = df_base['CC'].str.zfill(8)

df_base['CLAVSERV'] = df_base['CLAVSERV'].astype(str).str.zfill(4)
df_base['CENTROPAGO'] = df_base['CENTROPAGO'].astype(str).str.zfill(4)

print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

In [14]:
#################### ASIGNACION ####################

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar base de asignación?', icon='warning')

if result == 'yes':
    df_asignacion = pd.read_excel(asignacion_path)
    print(df_asignacion.shape)
    df_asignacion.head(5)

root.destroy()

In [ ]:
df_asignacion[df_asignacion['CC']==25515504]

In [ ]:
#################### BACKUPS ####################

In [ ]:
df_base_test = df_base.copy()
print('Base', df_base_test.shape)

In [ ]:
df_asignacion_test = df_asignacion.copy()
print('Asignacion', df_asignacion_test.shape)

In [ ]:
base_count = df_base_test.shape[0]
base_count

In [ ]:
def actualizar_tipo_cartera(df_base_test: pd.DataFrame) -> pd.DataFrame:
    df_base_test['TIPO_CARTERA'] = np.where(
        df_base_test['TIPO_CARTERA'].eq('UNSECURED').any(), 'UNSECURED',
        np.where(
            ~df_base_test['TIPO_CARTERA'].eq('NULL').any(), 'SECURED', 
            np.where(
                ~df_base_test['TIPO_CARTERA'].eq('SECURED').any(), 'NULL', 'SECURED/NULL'
            )
        )
    )
    return df_base_test

In [ ]:
cols_asignacion = ['CC', 'CONTRATO', 'NOMBRE_CLIENTE', 'TIPO_CARTERA', 'TIPO_FONDO', 'CARTERA', 'AGENCIA', 'FLAG']
df_asignacion_test = df_asignacion_test[cols_asignacion]

df_asignacion_test['CC'] = df_asignacion_test['CC'].astype('Int64').astype(str).str.zfill(8)
df_asignacion_test['CONTRATO'] = df_asignacion_test['CONTRATO'].astype('Int64').astype(str).str.zfill(18)

df_base_test = df_base_test.merge(df_asignacion_test, on='CC', how='left')
df_base_test.sort_values(by=['CC', 'TIPO_CARTERA'], ascending=[True, False], inplace=True)

filtro = (df_base_test['FLAG'] != 1) & (df_base_test['FLAG'].notna())
df_base_filtro = df_base_test.loc[filtro].groupby('CC').apply(actualizar_tipo_cartera).reset_index(drop=True)

df_base_test = df_base_test.merge(df_base_filtro[['CC', 'TIPO_CARTERA']], on='CC', how='left', suffixes=('', '_y'))
df_base_test['TIPO_CARTERA_FINAL'] = df_base_test['TIPO_CARTERA_y'].fillna(df_base_test['TIPO_CARTERA'])

df_base_test.drop(columns=['TIPO_CARTERA', 'TIPO_CARTERA_y'], inplace=True)
df_base_test.rename(columns={'TIPO_CARTERA_FINAL': 'TIPO_CARTERA'}, inplace=True)
df_base_test.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'], keep='first', inplace=True)

df_base_test['TIPO_CARTERA'] = df_base_test['TIPO_CARTERA'].fillna('NULL')
df_base_test['TIPO_FONDO'] = df_base_test['TIPO_FONDO'].fillna('NULL')

df_base_test['FLAG'] = df_base_test['FLAG'].astype('Int64')
df_base_test['CONTRATO'] = df_base_test['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)

cols_base_test = ['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA']
df_base_test = df_base_test[cols_base_test]

print(df_base_test.shape)
print(df_base_test.dtypes)
df_base_test.head(5)

In [ ]:
if base_count == df_base_test.shape[0]:
    print('REGISTROS OK')
elif base_count > df_base_test.shape[0]:
    print('Se han perdido registros')
else:
    print('Se han duplicado registros: ', df_base_test.shape[0] - base_count)

In [ ]:
df_base_no_flag = df_base_test[df_base_test['FLAG'].isnull()]
no_flag_count = df_base_no_flag.shape[0]
print(no_flag_count)
df_base_no_flag.head(5)

In [ ]:
#################### MONOPRODUCTO ####################

In [ ]:
df_mono = df_base_test[df_base_test['FLAG'] == 1]
mono_count = df_mono.shape[0]
print(mono_count)
df_mono.head(5)

In [ ]:
#################### REACTIVA ####################
fondos_gobierno = ['REACTIVA', 'CRECER', 'FAE', 'REACTIVA_KST']

In [ ]:
df_reactiva = df_mono[(df_mono['TIPO_CARTERA'] == 'UNSECURED') & (df_mono['TIPO_FONDO'].isin(fondos_gobierno))]

df_reactiva['TIPO_PAGO'] = None
df_reactiva['TIPO_FONDO'] = df_reactiva['TIPO_FONDO'].replace({'REACTIVA_KST': 'REACTIVA'}, regex=True)

cols_reactiva = ['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'TIPO_PAGO', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_reactiva = df_reactiva[cols_reactiva]

df_reactiva.sort_values(by=['FECHA', 'CC'], inplace=True)
df_reactiva.reset_index(drop=True, inplace=True)
df_reactiva.to_excel(reactiva, index=False)

reactiva_count = df_reactiva.shape[0]
print(reactiva_count)
df_reactiva.head(5)

In [ ]:
#################### FIN REACTIVA ####################

In [ ]:
df_no_mono = df_mono[(df_mono['TIPO_CARTERA'] != 'UNSECURED')]
no_mono_count = df_no_mono.shape[0]
print(df_no_mono.shape)
df_no_mono.head(5)

In [ ]:
df_mono = df_mono[(df_mono['TIPO_CARTERA'] == 'UNSECURED') & (~df_mono['TIPO_FONDO'].isin(fondos_gobierno))]

cols_mono = ['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA']
df_mono = df_mono[cols_mono]

df_mono.sort_values(by=['FECHA', 'CC'], inplace=True)
df_mono.reset_index(drop=True, inplace=True)
df_mono.to_excel(monoproducto, index=False)

mono_count_final = df_mono.shape[0]
print(df_mono.shape)
df_mono.head(5)

In [ ]:
if mono_count == mono_count_final + reactiva_count + no_mono_count:
    print('MONOPRODUCTO OK')
else:
    print('MONOPRODUCTO ERROR')

In [ ]:
#################### MULTIPRODUCTO ####################

In [ ]:
df_multi = df_base_test[df_base_test['FLAG'] > 1]
multi_count = df_multi.shape[0]
print(multi_count)
df_multi.head(5)

In [ ]:
df_no_multi = df_multi[(df_multi['TIPO_CARTERA'] != 'UNSECURED')]
no_multi_count = df_no_multi.shape[0]
print(df_no_multi.shape)
df_no_multi.head(5)

In [ ]:
df_multi = df_multi[df_multi['TIPO_CARTERA'] == 'UNSECURED']

df_multi['CONTRATO'] = None
df_multi['TIPO_FONDO'] = None
df_multi['CARTERA'] = None
df_multi['TIPO_PAGO'] = None

cols_multi = ['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'TIPO_PAGO', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_multi = df_multi[cols_multi]

df_multi.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'], inplace=True)
df_multi.sort_values(by=['FECHA', 'FLAG', 'CC'], inplace=True)
df_multi.reset_index(drop=True, inplace=True)
df_multi.to_excel(multiproducto, index=False)

multi_count_final = df_multi.shape[0]
print(df_multi.shape)
df_multi.head(5)

In [ ]:
if multi_count == multi_count_final + no_multi_count:
    print('MULTIPRODUCTO OK')
else:
    print('MULTIPRODUCTO ERROR')

In [ ]:
#################### NO ENVIADOS ####################

In [ ]:
df_no_enviados = pd.concat([df_no_mono, df_no_multi, df_base_no_flag])
df_no_enviados['FLAG'] = df_no_enviados['FLAG'].astype('Int64').fillna(0)

df_no_enviados['CONTRATO'] = df_no_enviados.apply(lambda x: None if x['FLAG'] != 1 else x['CONTRATO'], axis=1)
df_no_enviados['TIPO_FONDO'] = df_no_enviados.apply(lambda x: None if x['FLAG'] != 1 else x['TIPO_FONDO'], axis=1)
df_no_enviados['FLAG'] = df_no_enviados['FLAG'].apply(lambda x: 'NE' if x == 0 else x)

df_no_enviados.sort_values(by=['FECHA', 'FLAG', 'CC'], inplace=True)
df_no_enviados.reset_index(drop=True, inplace=True)
df_no_enviados.to_excel(no_enviados, index=False)

print(df_no_enviados.shape)
print(df_no_enviados.dtypes)
df_no_enviados.head(5)

In [ ]:
no_eviados = no_mono_count + no_multi_count + no_flag_count
enviados = mono_count_final + multi_count_final + reactiva_count

print('No enviados:', no_eviados)
print('Enviados:', enviados)
print('Total:', no_eviados + enviados)

if base_count == enviados + no_eviados:
    print('REGISTROS OK')
else:
    print('REGISTROS ERROR')

In [ ]:
#################### FORMATO EXCEL ####################

In [ ]:
import openpyxl as op
from openpyxl.styles import Font, PatternFill, Alignment

def format_excel(file_path: str, validator: str) -> None:
    workbook = op.load_workbook(file_path)
    sheet = workbook.active
    
    # Definir estilos generales
    general_font = Font(name='Calibri', size=11)
    header_font = Font(name='Calibri', size=11, bold=True, color='FFFFFF')
    alignment_center = Alignment(horizontal='center', vertical='center')
    
    # Definir estilos específicos
    header_font_white = Font(name='Calibri', size=11, bold=True, color='000000')
    header_fill_blue = PatternFill(start_color='002060', end_color='002060', fill_type='solid')
    header_fill_yellow = PatternFill(start_color='FFD965', end_color='FFD965', fill_type='solid')
    header_fill_green = PatternFill(start_color='C4D79B', end_color='C4D79B', fill_type='solid')
    header_fill_orange = PatternFill(start_color='FABF8F', end_color='FABF8F', fill_type='solid')
    
    # Aplicar estilos generales a todas las celdas
    for row in sheet.iter_rows():
        for cell in row:
            cell.font = general_font
    
    # Aplicar estilos al encabezado (fila 1)
    for cell in sheet[1]:
        cell.font = header_font
        cell.alignment = alignment_center
    
    # Aplicar estilos específicos según el validador
    if validator == 'mono' or validator == 'multi' or validator == 'env' or validator == 'noenv':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 12):  # Columnas J-K
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(12, 16):  # Columnas L-O
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(16, 17):  # Columna P
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'multi_agencias':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 13):  # Columnas J-L
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(13, 16):  # Columnas M-O
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'react_agencias':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 13):  # Columnas J-L
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(13, 15):  # Columnas M-N
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'reactiva':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 13):  # Columnas J-L
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(13, 15):  # Columnas M-N
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(15, 16):  # Columna O
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    workbook.save(file_path)

format_excel(mono_path, 'mono')
format_excel(multi_path, 'multi_agencias')
format_excel(react_path, 'react_agencias')
format_excel(no_enviados_path, 'noenv')

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Abrir archivos?', icon='warning')
if result == 'yes':
    os.startfile(mono_path)
    os.startfile(multi_path)
    os.startfile(react_path)
    os.startfile(no_enviados_path)

root.destroy()

In [ ]:
print('Base Bagos: ', df_base.shape)
print('--------------------------------------------')
print('Monoproducto:', df_mono.shape)
print('Importe Monoproducto:', round(df_mono['IMPORTE'].sum(), 2))
print('--------------------------------------------')
print('Multiproducto:', df_multi.shape)
print('--------------------------------------------')
print('reactiva:', df_reactiva.shape)
print('--------------------------------------------')
print('no enviados:', df_no_enviados.shape)

In [ ]:
hora = datetime.now() + timedelta(hours=1)

minuto_ajustado = 30 if 0 < hora.minute <= 30 else 0
hora = hora.replace(minute=minuto_ajustado, second=0, microsecond=0)

if minuto_ajustado == 0:
    hora += timedelta(hours=1)

periodo = 'PM' if hora.hour >= 12 else 'AM'

hora_formateada = f'{hora.strftime('%H:%M')} {periodo}'
hora_formateada

In [ ]:
from multiproducto import CorreoMultiproducto

root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

if df_reactiva.shape[0] == 0:
    flag_reactiva = False
else:
    flag_reactiva = True

result = messagebox.askquestion('Confirmación', f'¿Enviar correo MULTIPRODUCTO {fecha}?', icon='warning')
if result == 'yes':
    CorreoMultiproducto(fecha, file_path, hora_formateada, flag_reactiva).enviar_correo()

root.destroy()

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar multiproducto agencias?', icon='warning')
if result == 'yes':
    df_1 = pd.read_excel(f'{file_path}/multi_agencias/RJ.xlsx', dtype={'CONTRATO': str})
    df_2 = pd.read_excel(f'{file_path}/multi_agencias/CLASA.xlsx', dtype={'CONTRATO': str})
    df_3 = pd.read_excel(f'{file_path}/multi_agencias/MORNESE.xlsx', dtype={'CONTRATO': str})
    
    df_1 = df_1[cols_multi]
    df_2 = df_2[cols_multi]
    df_3 = df_3[cols_multi]
    
    df_1['AGENCIA'] = 'ASESCOM RJ'
    df_2['AGENCIA'] = 'CLASA MORA'
    df_3['AGENCIA'] = 'MORNESE MORA'
    
    contratos_rj = df_1['CONTRATO'].notna().value_counts()[True]
    contratos_clasa = df_2['CONTRATO'].notna().value_counts()[True]
    contratos_mornese = df_3['CONTRATO'].notna().value_counts()[True]
    
    print('Contratos RJ:', contratos_rj)
    print('Contratos CLASA:', contratos_clasa)
    print('Contratos MORNESE:', contratos_mornese)
    print('Total:', contratos_rj + contratos_clasa + contratos_mornese)
    print('--------------------------------------------------------------------------------------------------------------------------------------------')
    
    df_agencias = pd.concat([df_1, df_2, df_3])
    df_agencias.dropna(subset=['CONTRATO'], inplace=True)
    df_agencias['CONTRATO'] = df_agencias['CONTRATO'].apply(
        lambda x: str(int(x)).replace(' ', '').zfill(18) 
        if 16 <= len(str(x).replace(' ', '')) <= 18 
        else None)
    df_agencias.dropna(subset=['CONTRATO'], inplace=True)
    df_agencias.reset_index(drop=True, inplace=True)
    
    df_multi = pd.read_excel(multi_path, dtype={'CC': str, 'CLAVSERV': str, 'CENTROPAGO': str, 'CONTRATO': str})
    df_multi['AGENCIA'] = 'NULL'
    
    print('Multiproducto Original:', df_multi.shape[0])
    print('Multiproducto Agencias:', df_agencias.shape[0])
    print('--------------------------------------------------------------------------------------------------------------------------------------------')
    
    df_multi_final = pd.concat([df_multi, df_agencias])
    
    df_multi_final['CC'] = df_multi_final['CC'].astype(str).replace(' ', '').astype('Int64').astype(str).str.zfill(8)
    df_multi_final.drop(columns=['TIPO_FONDO', 'CARTERA'], inplace=True)
    df_multi_final.sort_values(by=['CC', 'CONTRATO'], inplace=True)
    df_multi_final.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'], keep='first', inplace=True)
    df_multi_final.reset_index(drop=True, inplace=True)
    
    df_asignacion_test['CC'] = df_asignacion_test['CC'].astype('Int64').astype(str).str.zfill(8)
    df_asignacion_test['CONTRATO'] = df_asignacion_test['CONTRATO'].apply(lambda x: str(int(x)).replace(' ', '').zfill(18) if pd.notna(x) else x)
    
    df_multi_final = df_multi_final.merge(df_asignacion_test, on='CONTRATO', how='left')
    df_multi_final.rename(columns={'CC_x': 'CC'}, inplace=True)
    df_multi_final.drop(columns=['CC_y'], inplace=True)
    
    df_multi_final['CLAVSERV'] = df_multi_final['CLAVSERV'].astype(str).str.zfill(4)
    df_multi_final['CENTROPAGO'] = df_multi_final['CENTROPAGO'].astype(str).str.zfill(4)
    df_multi_final['CONTRATO'] = df_multi_final['CONTRATO'].fillna('NULL')
    df_multi_final['TIPO_FONDO'] = df_multi_final['TIPO_FONDO'].fillna('NULL')
    df_multi_final['CARTERA'] = df_multi_final['CARTERA'].fillna('NULL')
    df_multi_final['TIPO_CARTERA'] = df_multi_final['TIPO_CARTERA'].fillna('NULL')
    df_multi_final['AGENCIA_y'] = df_multi_final['AGENCIA_y'].fillna('NULL')
    
    df_multi_final['AGENCIA_y'] = df_multi_final.apply(
        lambda x: x['AGENCIA_y'] 
        if x['AGENCIA_y'] == x['AGENCIA_x'] else f'{x['AGENCIA_x']}/{x['AGENCIA_y']}', 
        axis=1)
    
    df_multi_final.drop(columns=['AGENCIA_x', 'FLAG_y', 'NOMBRE_CLIENTE_y'], inplace=True)
    df_multi_final.rename(columns={'AGENCIA_y': 'AGENCIA', 'FLAG_x': 'FLAG', 'NOMBRE_CLIENTE_x': 'NOMBRE_CLIENTE'}, inplace=True)
    
    cols_multi_final = ['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'TIPO_PAGO', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA']
    df_multi_final = df_multi_final[cols_multi_final]
    df_multi_final.sort_values(by=['FECHA', 'FLAG', 'CC'], inplace=True)
    df_multi_final.reset_index(drop=True, inplace=True)
    
    # No Enviados Multiproducto
    df_no_enviados_multi = df_multi_final[(df_multi_final['CONTRATO'] == 'NULL') | (df_multi_final['TIPO_CARTERA'] != 'UNSECURED')]
    df_no_enviados_multi['AGENCIA'] = df_no_enviados_multi.apply(lambda x: 'AGENCIA NO IDENTIFICA' if x['CONTRATO'] == 'NULL' else x['AGENCIA'], axis=1)
    df_no_enviados_multi.drop(columns=['TIPO_PAGO'], inplace=True)
    print('No enviados Multiproducto:', df_no_enviados_multi.shape[0])
    
    # Reactivas Multiproducto
    df_reactivas_multi = df_multi_final[(df_multi_final['TIPO_CARTERA'] == 'UNSECURED') & (df_multi_final['TIPO_FONDO'].isin(fondos_gobierno))]
    df_reactivas_multi.drop(columns=['NOMBRE_CLIENTE', 'TIPO_CARTERA'], inplace=True)
    print('Reactivas Multiproducto:', df_reactivas_multi.shape[0])
    
    # Total Multiproducto
    df_multi_final = df_multi_final[(df_multi_final['CONTRATO'] != 'NULL') & 
                                    (df_multi_final['TIPO_CARTERA'] == 'UNSECURED') & 
                                    (~df_multi_final['TIPO_FONDO'].isin(fondos_gobierno))]
    df_multi_final.drop(columns=['TIPO_PAGO'], inplace=True)
    df_multi_final.to_excel(multiproducto, index=False)
    print('Total Multiproducto:', df_multi_final.shape[0])
    
    #Total No Enviados
    df_no_enviados_final = pd.concat([df_no_enviados, df_no_enviados_multi])
    df_no_enviados_final['CONTRATO'] = df_no_enviados_final['CONTRATO'].fillna('NULL')
    df_no_enviados_final['TIPO_FONDO'] = df_no_enviados_final['TIPO_FONDO'].fillna('NULL')
    df_no_enviados_final['CARTERA'] = df_no_enviados_final['CARTERA'].fillna('NULL')
    df_no_enviados_final['TIPO_CARTERA'] = df_no_enviados_final['TIPO_CARTERA'].fillna('NULL')
    df_no_enviados_final['NOMBRE_CLIENTE'] = df_no_enviados_final['NOMBRE_CLIENTE'].fillna('NULL')
    df_no_enviados_final['AGENCIA'] = df_no_enviados_final['AGENCIA'].fillna('NULL')
    df_no_enviados_final['CLAVSERV'] = df_no_enviados_final['CLAVSERV'].apply(lambda x: str(int(x)).replace(' ', '').zfill(4))
    df_no_enviados_final.sort_values(by=['FECHA', 'FLAG', 'CC'], inplace=True)
    df_no_enviados_final.reset_index(drop=True, inplace=True)
    df_no_enviados_final.to_excel(no_enviados, index=False)
    
    #Total Reactiva
    df_reactiva = pd.read_excel(react_path, dtype={'CC': str, 'CLAVSERV': str, 'CENTROPAGO': str, 'CONTRATO': str})
    df_reactiva_final = pd.concat([df_reactiva, df_reactivas_multi])
    df_reactiva_final['TIPO_PAGO'] = df_reactiva_final['TIPO_PAGO'].str.upper().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    df_reactiva_final['TIPO_PAGO'] = df_reactiva_final['TIPO_PAGO'].replace(' ', '').replace('TOTAL', 'CANCELACION').replace('PARCIAL', 'AMORTIZACION')
    df_reactiva_final.to_excel(reactiva, index=False)
    
    # Total Enviados
    df_enviados = pd.concat([df_mono, df_multi_final])
    df_enviados['CLAVSERV'] = df_enviados['CLAVSERV'].apply(lambda x: str(int(x)).zfill(4))
    df_enviados['CENTROPAGO'] = df_enviados['CENTROPAGO'].apply(lambda x: str(int(x)).zfill(4))
    df_enviados.sort_values(by=['FECHA', 'FLAG', 'CC'], inplace=True)
    df_enviados.reset_index(drop=True, inplace=True)
    df_enviados.to_excel(si_enviados, index=False)
    
    print('-------------------------------------------- REPORTE FINAL --------------------------------------------')
    print('Base Bagos: ', df_base.shape)
    print('--------------------------------------------')
    print('Monoproducto:', df_mono.shape)
    print('Importe Monoproducto:', round(df_mono['IMPORTE'].sum(), 2))
    print('--------------------------------------------')
    print('Multiproducto:', df_multi_final.shape)
    print('Importe Multiproducto:', round(df_multi_final['IMPORTE'].sum(), 2))
    print('--------------------------------------------')
    print('Reactiva:', df_reactiva_final.shape)
    print('Importe Reactiva:', round(df_reactiva_final['IMPORTE'].sum(), 2))
    print('--------------------------------------------')
    print('No Enviados:', df_no_enviados_final.shape)
    
    format_excel(multi_path, 'multi')
    format_excel(react_path, 'reactiva')
    format_excel(no_enviados_path, 'noenv')
    format_excel(enviados_path, 'env')
    
    os.startfile(multi_path)
    os.startfile(react_path)
    os.startfile(no_enviados_path)
    os.startfile(enviados_path)

root.destroy()

In [ ]:
lista_cc = [
    "23207671", "28687962", "20423547", "20423547", "13253110", "20218217", "29089634", "30435313", "30435313", "31261027",
    "23589142", "22347136", "25630118", "26491018", "20078513", "20778487", "21665261", "22037547", "22831243", "23676300",
    "25404000", "25404979", "25404979", "25453527", "26097854", "26254491", "26297016", "26297016", "26805049", "27129908",
    "27663335", "27672360", "27701976", "27861643", "28315754", "28419117", "30038930", "30802477", "31014568", "21524509",
    "23616992", "25874888", "26476829", "04295145", "14171150", "14982272", "21215143", "21306331", "21524509", "24396287",
    "25492993", "02373823", "02373823", "23737569", "25711963", "29183914", "04245822", "21250861", "22047268", "23241819",
    "23571118", "24528159", "29201244", "26592561", "26731386", "24465390"
]

df_asignacion_test[
    (df_asignacion_test['CC'].isin(lista_cc)) & 
    (df_asignacion_test['AGENCIA'].isin(['CLASA MORA', 'SIN AGENCIA'])) & 
    (df_asignacion_test['CARTERA']=='EXJ') & 
    (df_asignacion_test['FLAG']!=1)
]

In [ ]:
df_asignacion_test[df_asignacion_test['CC']=='20218217']

In [ ]:
df_asignacion_test[df_asignacion_test['CONTRATO']=='001102519600160144']